# 8. Setting up scripts to run MCMC 

This is just a notebook for testing various bits of functionality in the functions included in the SCFC functions, and for documenting what everything does. 

In [1]:
datafile = '/Users/Megan/RajLab/BRAIN/SCFC-spectral-python/SCFC/data/downsampled_patients/freqMEGdata_8002-101.h5'

OK, let's try to put together a functioning complete script to run MCMC in a parallel way, and then add in functionality.

The script needs to include following steps:

- import the necessary modules
- load the patient data
- include the parameters of the prior distributions (which can be altered) in the prior param_dict
- create an instance of the Brain class on which the simulation can be run
- set up a starting position for the sampler, and the sampler variables: number of walkers and run steps
- set up the sampler object itself
- run 
- save out the data

Ideally, we want to build in functionality to run this in a parallelized way, and to save out data as we go. This script is in **scripts/runMCMC.py**.

Import generic modules and the required parts of the SCFC:

In [20]:
#generic modules
import sys, os
sys.path.append("..")
import matplotlib.pyplot as mpl
%matplotlib inline
from scipy.io import loadmat
import numpy as np
import emcee
from multiprocessing import Pool

In [9]:
from utils import path 
from brain import Brain

Import the data

In [8]:
FMEGdownsample = path.read_hdf5(datafile)

Create an instance of Brain class, and set up the correct corresponding frequency vector.

In [10]:
fmin = 2 # 2Hz - 45Hz signal range
fmax = 45
fvec = np.linspace(fmin,fmax,40)

In [12]:
thisbrain = Brain.Brain()
hcp_dir = path.get_sibling_path('data') # connectome information is in /data/ dir
thisbrain.add_connectome(hcp_dir) # Use default files in /data/
thisbrain.reorder_connectome(thisbrain.connectome, thisbrain.distance_matrix)
thisbrain.bi_symmetric_c()
thisbrain.reduce_extreme_dir()

Set up a starting array of the parameters. Remember they are ordered according to the list in 'debug-MCMC' and 'prior-distributions': 

- tau_e
- tau_i
- alpha
- speed
- gei
- gii
- tauC

These starting positions are at the discretion of the user. Here, I used fitted simulated annealing parameters for one of the patients from the healthy cohort, but deliberately set the time constant parameters to be very high so that a burn-in period can be observed.

In [13]:
param_start = [0.2,
 0.2,
 0.9670233044667013,
 14.479055812222878,
 3.9552727546221096,
 1.7950301129315296,
 0.2]

Set up the number of dimensions and number of walkers (nwalkers is a variable that needs to be adjusted to optimise MCMC).

Set up a starting position in the 7-dimensional space.

In [14]:
ndim, nwalkers = 7, 20

In [15]:
pos = [np.transpose(param_start) + 0.5e-3*np.random.randn(ndim) for i in range(nwalkers)] #starting position in parameter space.

Set up the parallelizer pool. 